In [6]:
from myUtils import pickle_load
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
filename_pickled_df_symbols_close = "df_symbols_close"  # pickled filename

In [7]:
# load df_symbols_close
df = pickle_load(path_data_dump, filename_pickled_df_symbols_close)
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_orig = df.copy()  # backup df
df = df.tail(252)  # keep only the last 252 days of data
symbols_before_dropna = df.columns
shp_before = df.shape

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close


In [20]:
df = df.dropna(axis=1, how='any')  # drop symbols (i.e. columns) with NaN
symbols_after_dropna = df.columns
symbols_dropped = symbols_before_dropna.difference(symbols_after_dropna).tolist()
# print(f'symbols with NaN dropped from dataframe :{symbols_dropped}')
print(f'{len(symbols_dropped)} symbols have NaN in their close price, and were dropped from the dataframe.\n')
print(f'These symbols were dropped:\n{symbols_dropped}\n')
shp_after = df.shape
print(f'                      rows     columns')
print(f'df before dropna  {shp_before[0]:>8}    {shp_before[1]:>8}')
print(f'df after dropna   {shp_after[0]:>8}    {shp_after[1]:>8}')

49 symbols have NaN in their close price, and were dropped from the dataframe.

These symbols were dropped:
['ACT', 'BF-A', 'BLCO', 'BRCC', 'BRZE', 'BXSL', 'CEG', 'CMTG', 'CRDO', 'CRGY', 'CWAN', 'DH', 'EE', 'ESAB', 'ESMT', 'FLNC', 'FRSH', 'FYBR', 'GBTG', 'GFS', 'GOOG', 'GTLB', 'HCP', 'HKD', 'HLN', 'IHS', 'INFA', 'IOT', 'KD', 'LTH', 'NU', 'OLPX', 'ONON', 'PFHC', 'RIVN', 'RTLR', 'SLVM', 'SMCI', 'SMR', 'SNAP', 'SNOW', 'SRAD', 'STER', 'TCN', 'TOST', 'TWKS', 'UDMY', 'VALE', 'WE']

                      rows     columns
df before dropna       252        1917
df after dropna        252        1868


In [21]:
from myUtils import symb_perf_stats_vectorized_v1
symbols, period_yr, drawdown, UI, max_drawdown, \
        returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = symb_perf_stats_vectorized_v1(df)

In [27]:
print(f'drawdown(rows, columns): {drawdown.shape[0]}, {drawdown.shape[1]}')
print(f'type(drawdown): {type(drawdown)}')

drawdown(rows, columns): 252, 1868
type(drawdown): <class 'numpy.ndarray'>


In [37]:
import numpy as np
from myUtils import nan_pad, moving_window, UI_MW
UIMW15 = np.apply_along_axis(lambda x: UI_MW(x, window=15), 0, drawdown)
UIMW30 = np.apply_along_axis(lambda x: UI_MW(x, window=30), 0, drawdown)
print(f'UIMW15.shape : {UIMW15.shape}')
print(f'UIMW30.shape : {UIMW30.shape}')

UIMW15.shape : (252, 1868)
UIMW30.shape : (252, 1868)


In [ ]:
anr = abs(1 - UIMW30/UIMW15)  # absolute normalized ratio  
anr.shape

In [ ]:
anr15 = anr[-15:, :]
anr15

In [ ]:
fMask = np.where((anr15 < .0005), True, False)  # filtered masked 2d np.array
idx_filtr_anr15 = np.column_stack(np.where(fMask))
# len(idx_filtr_anr15)

In [ ]:
idx_syms = idx_filtr_anr15[:,1].tolist()
idx_syms = list(set(idx_syms))
idx_syms.sort()

In [ ]:
fSyms = []
for idx in idx_syms:
  fSyms.append(df.columns[idx])
len(fSyms)

In [ ]:
['RPRX', 'SNA', 'VRTX']

In [ ]:
import yfinance as yf
from myUtils import yf_print_symbol_data, yf_candlestick
# plot symbols
plot_days = '252d'
for symbol in fSyms[80:95]:
    # yf.Ticker OHLC are adjusted correctly
    df = yf.Ticker(symbol).history(period=plot_days)
    df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)  # drop Div. and Stock Splits columns
    df.columns = ['open', 'high', 'low', 'close', 'volume']  # rename columns Open ... Volume to open ... volume
    cache_plot = yf_candlestick(symbol, df, plot_chart=True)